In [0]:
import pandas as pd

data = pd.read_csv('/tmp/eider-user/userfile/nayakams/AirPassengers.csv')
print(data.head(),data.dtypes)

In [0]:
dateparser = lambda date: pd.datetime.strptime(date, '%Y-%m')
data = pd.read_csv('/tmp/eider-user/userfile/nayakams/AirPassengers.csv', parse_dates = ['Month'], index_col = 'Month', date_parser = dateparser)
data.head()
data.dtypes


In [0]:
#Checking Stationarity

from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

def test_stationarity(ts):
    
    #determining rolling statistics
    rolmean = pd.rolling_mean(ts, window = 12)
    rolstd = pd.rolling_std(ts, window = 12)
    
    #plotting rolling stats
    
    plt.plot(ts, color = 'black', label = 'original')
    plt.plot(rolmean, color = 'blue', label = 'rol_avg')
    plt.plot(rolstd, color = 'red', label = 'rol_std')
    plt.show()
    
    #performing dfuller test
    
    dftest = adfuller(ts.iloc[:,0], autolag = 'AIC')
    print(dftest)

    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#lag values', 'Number of observations used'])
    
    for key, values in dftest[4].items():
        dfoutput['Critical value (%s)' %key] = values
    print(dfoutput)
    
test_stationarity(data)
    

In [0]:
#To peanalise hihger values we can use log, sqaureroot, cube root and this process is called transformation.
import numpy as np
logdf = np.log(data)
plt.plot(logdf)


In [0]:
#Eliminating Trend and Seasonality.
#This can be done by 2 Methods: 1.Differencing  2.Decomposing

#Differencing

diffdata = logdf - logdf.shift()
diffdata.dropna(inplace = True)

test_stationarity(diffdata)

In [0]:
#Decomposing

from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(logdf)

seasonal = decomposition.seasonal
trend = decomposition.trend
residual = decomposition.resid

plt.subplot(411)
plt.plot(logdf, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend()
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [0]:
# Trend and Seasonality are seperated from data. Hence we can model the residuals

residual.dropna(inplace = True)
test_stationarity(residual)

In [0]:

# Forecasting Time Series

#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf


lag_acf = acf(diffdata, nlags=20)
lag_pacf = pacf(diffdata, nlags=20, method='ols')

#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(diffdata)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(diffdata)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')


#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(diffdata)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(diffdata)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [0]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(logdf, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(diffdata)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues - diffdata)**2))

In [0]:
#Taking back to original scale

predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print predictions_ARIMA_diff.head()

In [0]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print predictions_ARIMA_diff_cumsum.head()

In [0]:
predictions_ARIMA_log = pd.Series(logdf.ix[:,0], index=logdf.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [0]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(data)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-data)**2)/len(data)))